In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### First let us import all the required packages for our analysis and create an Import class for calling it.

In [ ]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt
from datetime import datetime
from glob import glob
    
import logging
logger = logging.getLogger('simple_example')
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)
logger.info("Completed importing all the required packages")

In [ ]:
os.chdir('/kaggle/input/stroke-prediction-dataset')
files=glob(os.getcwd()+"/*.csv")
for file in files:
    input_data=pd.read_csv(file)

In [ ]:
input_data.head()

### There are 11 independant variables and 1 dependant variable in our analysis.

### Out of the independant variables age,avg_gluscose_level and bmi are continuous numerical variables and the remaining variables like gender, hypertension (1 : Yes, 0 : No), heart_disease (1 : Yes, 0 : No), ever_married, work_type, residence_type, smoking_status are discrete categorical variables.

### NOTE : id won't add much significance to our study so we would remove it from our analysis

### The dependent variable stroke is a discrete categorical variable     (1 : Yes, 0 : No)


In [ ]:
def drop_variable(df,columns):
    for col in columns:
        df.drop(columns=col,axis=1,inplace=True)
        return df

In [ ]:
input_data=drop_variable(input_data,["id"])

In [ ]:
def get_na_perc(df):
    print(np.round(df.isna().sum() / df.shape[0] * 100.0,2))

In [ ]:
get_na_perc(input_data)

### Only 3.93 % of bmi values are missing from our analysis. We would take 3 approaches to tackle this

#### 1) Imput bmi with its mean value
#### 2) Remove the rows having missing bmi from our analysis
#### 3) Remove the bmi column altogether

### Imputing bmi with mean value

In [ ]:
def impute_with_mean(df,columns):
    for col in columns:
        df[col]=df[col].fillna(df[col].mean())
        return df

In [ ]:
input_data=impute_with_mean(input_data,["bmi"])

### No let us check whether the missing values of bmi are imputed properly or not

In [ ]:
get_na_perc(input_data)

### Let us perform one hot encoding for the categorical valriables

In [ ]:
def perform_one_hot_encode(df,cols):
    df=pd.get_dummies(df,columns=cols,prefix_sep="_")
    return df

In [ ]:
input_data=perform_one_hot_encode(input_data,["gender","hypertension","heart_disease","ever_married","work_type","Residence_type","smoking_status"])

In [ ]:
input_data.head()

### Now let us normalize the numerical variables so that they are all on the same scale

In [ ]:
from sklearn.preprocessing import StandardScaler
def standardise_numerical_variables(df,columns):
    for col in columns:
        x=np.array(df[col]).reshape(-1,1)
        scaler=StandardScaler()
        df[col]=scaler.fit_transform(x)
    return df

In [ ]:
input_data=standardise_numerical_variables(input_data,["age","avg_glucose_level","bmi"])

### Let us check for the distribution of stroke patients in our analysis

In [ ]:
input_data["stroke"].value_counts()

### Most of the patients in our analysis have stroke : 0 which signify that they have not suffered from stroke. So we must use SMOTETomek to account for this imbalance in the distribution of our dependant variable

In [ ]:
y=input_data["stroke"]
X=input_data.drop(columns=["stroke"],axis=1)

In [ ]:
from imblearn.combine import SMOTETomek
tomek=SMOTETomek()

X_res,y_res=tomek.fit_resample(X,y)

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X_res,y_res,test_size=0.25,random_state=0)

### Performance Measure using train-test split

In [ ]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression()

logit.fit(X_train,y_train)

y_pred=logit.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

### Perfromance measure using Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score,cross_validate

from sklearn.linear_model import LogisticRegression
logit = LogisticRegression()

from sklearn.metrics import classification_report, accuracy_score, make_scorer

def classification_report_with_accuracy_score(y_true, y_pred):
    print(classification_report(y_true, y_pred)) # print classification report
    return accuracy_score(y_true, y_pred) # return accuracy score

#nested_score = cross_val_score(logit, X=X_res, y=y_res,scoring=make_scorer(classification_report_with_accuracy_score(y_test,y_pred)))

cv_results = cross_validate(logit, X_res,y_res, cv=10) #cv=10 indicates 10 fold cross validation
print(cv_results['test_score'].mean())

### Compare Different ML models using cross validation

In [ ]:
def cv_comparison(models,X,y,cv):
    cv_accuracies = pd.DataFrame()
    accs = []
    pres = []
    recs = []
    f1s = []
    
    for model in models:
        acc = cross_val_score(model, X, y, scoring='accuracy', cv=cv)
        accs.append(acc)
        pre = cross_val_score(model, X, y, scoring='precision', cv=cv)
        pres.append(pre)
        rec = cross_val_score(model, X, y, scoring='recall', cv=cv)
        recs.append(rec)
        f1 = cross_val_score(model, X, y, scoring='f1', cv=cv)
        f1s.append(f1)
        cv_accuracies[str(model)] = [acc.mean(), pre.mean(), rec.mean(), f1.mean()]
    cv_accuracies.index = ['Accuracy', 'Precision', 'Recall', 'F1']
    return cv_accuracies, accs, pres, recs, f1s

In [ ]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression()

from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()

from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier()

from sklearn.svm import SVC
svc = SVC()

models=[logit,tree,forest,svc]

cv_df,accs, pres, recs, f1s = cv_comparison(models,X_res,y_res,10)

In [ ]:
cv_df